In [6]:
# --- Notebook bootstrap: locate backend, fix sys.path, ensure packages ---
import sys
from pathlib import Path

def find_backend_root(start: Path, look_for="app") -> Path:
    cur = start.resolve()
    for _ in range(10):
        if (cur / look_for).is_dir():
            return cur
        cur = cur.parent
    raise RuntimeError(f"Couldn't find '{look_for}/' starting from {start}")

# 1) Find backend root (directory that contains app/)
BACKEND_ROOT = find_backend_root(Path.cwd(), "app")

# 2) Ensure package markers exist (safe if already present)
for pkg in [BACKEND_ROOT / "app", BACKEND_ROOT / "app" / "db", BACKEND_ROOT / "app" / "db" / "operations"]:
    (pkg / "__init__.py").touch(exist_ok=True)

# # 3) Put backend root on import path
if str(BACKEND_ROOT) not in sys.path:
    sys.path.insert(0, str(BACKEND_ROOT))

# 4) Quick sanity check
import importlib
app_mod = importlib.import_module("app")
print("app loaded from:", app_mod.__file__)


app loaded from: C:\Users\danse\Projects\FleetAI V1\fleet-ai\apps\backend\app\__init__.py


In [10]:
from app.db.session import AsyncSessionLocal
from app.db.operations.airports import get_airports_by_org

# Pick an actual org_id from your DB
org_id = "6c26bb58-b361-4feb-aa6b-1f01599200c5"

async with AsyncSessionLocal() as session:
  airports = await get_airports_by_org(session, org_id)

for i, airport  in enumerate(airports):
    print(i, airport)

selected_airport = airports[0]
name = selected_airport.name
city = selected_airport.city
country = selected_airport.country
code = selected_airport.iata
print(f"\n{name}, {city}, {country} ({code})")


2025-08-27 10:26:14,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-08-27 10:26:14,960 INFO sqlalchemy.engine.Engine SELECT airports.id, airports.org_id, airports.name, airports.country, airports.created_at, airports.updated_at, airports.iata, airports.icao, airports.city, airports.state, airports.is_hub, airports.internal_notes 
FROM airports 
WHERE airports.org_id = $1::UUID
2025-08-27 10:26:14,961 INFO sqlalchemy.engine.Engine [cached since 77.4s ago] ('6c26bb58-b361-4feb-aa6b-1f01599200c5',)
2025-08-27 10:26:14,963 INFO sqlalchemy.engine.Engine ROLLBACK
0 <app.db.models_auto.Airports object at 0x0000015F084F71D0>
1 <app.db.models_auto.Airports object at 0x0000015F084F7B90>
2 <app.db.models_auto.Airports object at 0x0000015F084F7A10>
3 <app.db.models_auto.Airports object at 0x0000015F084F77D0>
4 <app.db.models_auto.Airports object at 0x0000015F084F6810>
5 <app.db.models_auto.Airports object at 0x0000015F084F7AD0>
6 <app.db.models_auto.Airports object at 0x0000015F084F7C50>

